In [1]:
import os
import sys
import random
import string
import json

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

# Check versions of libraries
print(f"Python: {sys.version}")
print(f"numpy: {np.__version__}")
print(f"tensorflow: {tf.__version__}")

print(tf.config.list_physical_devices("GPU"))

Python: 3.8.10 (default, May 19 2021, 18:05:58) 
[GCC 7.3.0]
numpy: 1.19.5
tensorflow: 2.5.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4608)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
tf.debugging.set_log_device_placement(True)

1 Physical GPUs, 1 Logical GPUs


In [3]:
CURRENT_DIR = os.getcwd()
TRAINING_SET_DIR = os.path.join(CURRENT_DIR, "images", "training_set")
VALIDATION_SET_DIR = os.path.join(CURRENT_DIR, "images", "validation_set")
ANNOTATIONS_DIR = os.path.join(CURRENT_DIR, "annotations")
LABELMAP_DIR = os.path.join(ANNOTATIONS_DIR, "label_map.pbtxt")
TFRECORDS_TRAINING_DIR = os.path.join(ANNOTATIONS_DIR, "train.record")
TFRECORDS_VALIDATION_DIR = os.path.join(ANNOTATIONS_DIR, "test.record")
TFRECORDS_SCRIPT_DIR = os.path.join(CURRENT_DIR, "scripts", "generate_tfrecord.py")
LOGS_DIR = os.path.join(CURRENT_DIR, "models", "efficientdet_d0_coco17_tpu-32", "logs")

EPOCHS = 15
BATCH_SIZE = 24
NUMBER_OF_TRAINING_IMAGES = 0
NUMBER_OF_VALIDATION_IMAGES = 0

# Must be the same input size as the base model
# Currently using: EfficientNetB5 - (456, 456)
IMAGE_WIDTH = 456
IMAGE_HEIGHT = 456

In [ ]:
# Creates Label Map file at label_map.pbtxt in the annotations directory
breeds = os.listdir(TRAINING_SET_DIR)
labels = []

for index, label in enumerate(breeds):
    labels.append({"id":index+1, "name":label.rstrip("0123456789")[:-2]})

with open(LABELMAP_DIR, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
#!python {TFRECORDS_SCRIPT_DIR -x TRAINING_SET_DIR -l LABELMAP_DIR -o TFRECORDS_TRAINING_DIR}
#!python {TFRECORDS_SCRIPT_DIR -x VALIDATION_SET_DIR -l LABELMAP_DIR -o TFRECORDS_VALIDATION_DIR}

In [4]:
!python model_main_tf2.py --model_dir=models/efficientdet_d0_coco17_tpu-32 \
    --pipeline_config_path=models/efficientdet_d0_coco17_tpu-32/pipeline.config \
    --alsologtostderr

2021-07-13 04:19:54.713544: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-13 04:19:56.464513: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-13 04:19:56.489283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-13 04:19:56.489828: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:26:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.68GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2021-07-13 04:19:56.489873: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-13 04:19:56.491589: I tensorflow/stream_executor/pl

I0713 04:19:56.972373 140093856380736 efficientnet_model.py:147] round_filter input=16 output=16
I0713 04:19:56.972463 140093856380736 efficientnet_model.py:147] round_filter input=24 output=24
I0713 04:19:57.099083 140093856380736 efficientnet_model.py:147] round_filter input=24 output=24
I0713 04:19:57.099184 140093856380736 efficientnet_model.py:147] round_filter input=40 output=40
I0713 04:19:57.244260 140093856380736 efficientnet_model.py:147] round_filter input=40 output=40
I0713 04:19:57.244361 140093856380736 efficientnet_model.py:147] round_filter input=80 output=80
I0713 04:19:57.452590 140093856380736 efficientnet_model.py:147] round_filter input=80 output=80
I0713 04:19:57.452690 140093856380736 efficientnet_model.py:147] round_filter input=112 output=112
I0713 04:19:57.659434 140093856380736 efficientnet_model.py:147] round_filter input=112 output=112
I0713 04:19:57.659541 140093856380736 efficientnet_model.py:147] round_filter input=192 output=192
I0713 04:19:57.933911 14